# Meter Extraction
Goal: Extract distinct meters and relevant attributes from PVACD Access DB

In [1]:
import pyodbc
import pandas as pd

### Load meter types and mapping from CSV
This is used to assign each imported meter a meter type ID

In [2]:
#Load CSVs from the data folder
metertypes = pd.read_csv('./api/data/devdata_metertypes.csv')
oldmetertypes = pd.read_csv('./api/data/devdata_metertypemap.csv')

In [3]:
#Merge metertypes and metermap based on brand and model_number
metertypemap = oldmetertypes.merge(metertypes, left_on=['sBrand', 'newModel'], right_on=['brand','model_number'], how='left')
metertypemap.head(10)

,sSize,sBrand,sModel,newModel,id,brand,series,model_number,size,description
0,NaN,NaN,"1""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,NaN,308,MO308,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,BADGER,"2""","2""",NaN,NaN,NaN,NaN,NaN,NaN
3,8.0,BADGER,"1""","1""",NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,BADGER,"1""","1""",NaN,NaN,NaN,NaN,NaN,NaN
5,1.5,BADGER,1.5,T-0160,21.0,BADGER,NaN,T-0160,1.500,1.5 TURBO W/BOLT KITS & FLGS
6,3.0,BADGER,"3""","3""",18.0,BADGER,NaN,"3""",3.000,"3"" CHAMBER"
7,3.0,BADGER,3IN,"3""",18.0,BADGER,NaN,"3""",3.000,"3"" CHAMBER"
8,6.0,BADGER,3/4 IN,"3/4""",19.0,BADGER,NaN,"3/4""",0.750,BADGER W / CLPGS
9,4.0,BADGER,5/8,"5/8""",20.0,BADGER,NaN,"5/8""",0.625,BADGER W / CLPGS


In [5]:
#Test filtering
test = metertypemap[(metertypemap['sBrand'] == 'BADGER') & (metertypemap['sSize'] == 1.5) & (metertypemap['sModel'] == '1.5')]
test

,sSize,sBrand,sModel,newModel,id,brand,series,model_number,size,description
5,1.5,BADGER,1.5,T-0160,21.0,BADGER,NaN,T-0160,1.5,1.5 TURBO W/BOLT KITS & FLGS


In [8]:
test.iloc[1,4]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
# A function that maps an input size, brand, and model to the meter type ID
def getMeterType(size, brand, model):
    # Filter the meter type map to only include the size, brand, and model number
    filtered = metertypemap[(metertypemap['size'] == size) & (metertypemap['sBrand'] == brand) & (metertypemap['sModel'] == model)]
    # If there is a match, return the meter type ID
    if len(filtered) > 0:
        return filtered['meter_type_id'].values[0]
    # Otherwise, return None
    else:
        return None

In [ ]:
#Set up PyODBC connection to Access database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\clmbn\NMWaterData\PVACDdb\PVACD AMS Jan2023.accdb;'
    )
cnxn = pyodbc.connect(conn_str)

### Data Import
Source: 
tblMeter - It appears that tblMeter is mostly tracking maintenance while tblMeter is mostly unique meter serial numbers.

Fields:
- sTag, sSize, sBrand, sContact, sSerialNumber, sRANumber, sTRSS, sLatLong, sModel, sStatus

In [ ]:
qry = '''
    SELECT DISTINCT sSerialNumber, sSize, sBrand, sModel, sContact, sTag, sRANumber, sTRSS, sLatLong, sStatus 
    FROM tblMeter
    WHERE sSerialNumber IS NOT NULL AND sSerialNumber <> ''
'''
meter_df = pd.read_sql_query(qry, cnxn)
meter_df.head()

### Data Cleaning
- Remove duplicates

In [ ]:
#Drop duplicate serial numbers from meter_df
meter_df.drop_duplicates(subset='sSerialNumber', keep='first', inplace=True)

### Prepare data for new database